### QGIS CSV merges per PHU
**Authors:** Sofia and KT

In [1]:
import os
import numpy as np
import pandas as pd
import glob

---
### Import and Clean Data


In [21]:
os.chdir('../data/QGIS_csv_files/')

FileNotFoundError: [Errno 2] No such file or directory: '../data/QGIS_csv_files/'

In [190]:
# read data
prox = pd.read_csv('../PMD-en/PMD-on.csv')
qgis = pd.read_csv('Joined_DB_to_PHU.csv')
covid = pd.read_csv('../covid_comorbidities.csv')

In [191]:
# only keep necessary columns
qgis = qgis[['fid', 'DBUID', 'PRUID', 'PRNAME', 'CSDUID', 'CSDNAME', 'CSDTYPE', 'CMAUID', 'CMAPUID', 
     'CMANAME', 'CMATYPE', 'HR_UID', 'ENG_LABEL', 'FRE_LABEL']]

In [276]:
prox = pd.read_csv('../PMD-en/PMD-on.csv')

In [285]:
# remove rows that don't have prox data ~ 141 rows removed
prox2 = prox[prox['prox_idx_emp'] != 'F']

# convert .. to np.nan
list_prox = ['prox_idx_emp', 'prox_idx_pharma', 'prox_idx_childcare',
            'prox_idx_health', 'prox_idx_grocery', 'prox_idx_educpri',
            'prox_idx_educsec', 'prox_idx_lib', 'prox_idx_parks',
            'prox_idx_transit']

for each in list_prox:
    prox2[each] = prox2[each].replace({"..":np.nan})

# convert to float
prox2[list_prox] = prox2[list_prox].astype(float)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [286]:
prox2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132711 entries, 0 to 133213
Data columns (total 41 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   DBUID               132711 non-null  int64  
 1   DBPOP               132602 non-null  object 
 2   DAUID               132711 non-null  int64  
 3   DAPOP               132602 non-null  object 
 4   CSDUID              132711 non-null  int64  
 5   CSDNAME             132711 non-null  object 
 6   CSDTYPE             132711 non-null  object 
 7   CSDPOP              132602 non-null  object 
 8   CMAUID              97533 non-null   float64
 9   CMAPUID             97533 non-null   float64
 10  CMANAME             97533 non-null   object 
 11  CMATYPE             97533 non-null   object 
 12  CMAPOP              97533 non-null   object 
 13  PRUID               132711 non-null  int64  
 14  PRNAME              132711 non-null  object 
 15  PRPOP               132711 non-nul

---
### Rename QGIS data HRs to PHUs given by Ontario Case Data

Reference `comorbidities-merges.ipynb` for justifications.

In [287]:
# QGIS
qgis.rename(columns = {'ENG_LABEL' : 'Reporting_PHU'}, inplace = True)

qgis.Reporting_PHU.replace({'York Regional Health Unit' : 'York Region Public Health Services',
              'Waterloo Health Unit' : 'Region of Waterloo, Public Health',
            'Halton Regional Health Unit' : 'Halton Region Health Department',
              'Lambton Health Unit' : 'Lambton Public Health',
             'Durham Regional Health Unit' : 'Durham Region Health Department',  
             'City of Ottawa Health Unit' : 'Ottawa Public Health', 
             'Peel Regional Health Unit': 'Peel Public Health',
              'Wellington-Dufferin-Guelph Health Unit': 'Wellington-Dufferin-Guelph Public Health',
            'Kingston, Frontenac and Lennox and Addington Health Unit' : 'Kingston, Frontenac and Lennox & Addington Public Health',
             'The Eastern Ontario Health Unit':'Eastern Ontario Health Unit',
              'City of Hamilton Health Unit':'Hamilton Public Health Services',
              'Niagara Regional Area Health Unit':'Niagara Region Public Health Department',
              'Sudbury and District Health Unit':'Sudbury & District Health Unit',
             'The District of Algoma Health Unit':'Algoma Public Health Unit',
             'Peterborough County - City Health Unit':'Peterborough Public Health',
             'City of Toronto Health Unit' : 'Toronto Public Health',
              'Perth District Health Unit' : 'Huron Perth District Health Unit',
            'Huron County Health Unit' : 'Huron Perth District Health Unit',
            'Elgin-St. Thomas Health Unit' : 'Southwestern Public Health',
            'Oxford County Health Unit' : 'Southwestern Public Health'}, inplace = True)

In [288]:
# merge
merge = pd.merge(qgis, prox3, how = 'inner', on = 'DBUID')

In [289]:
# calc median of proximity per PHU
phu_med = merge.groupby('Reporting_PHU').median()

In [290]:
# merge with covid
pd.merge(covid, phu_med, how = 'outer', on = 'Reporting_PHU').to_csv('../PMD-en/PHU_FINAL.csv')